In [1]:
#Importing libraries for data manipulation
import pandas as pd
import numpy as np
import re,string,random
import torchtext
import torch
from torchtext import data

#For preprocessing of text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.data.utils import get_tokenizer

#For building classifier
from sklearn.model_selection import train_test_split

# Dictionary is taken from kaggle
contractions_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                    "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  
                    "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                    "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", 
                    "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                    "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                    "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                    "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                    "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
                    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                    "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                    "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is",
                    "that'd": "that would", "that'd've": "that would have", "that's": "that is",
                    "there'd": "there would", "there'd've": "there would have", "there's": "there is", 
                    "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are",
                    "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                    "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", 
                    "what'll've": "what will have", "what're": "what are",  "what's": "what is", 
                    "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", 
                    "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not",
                    "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                    "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                    "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def convert_contraction_to_original(tweet):
    if type(tweet) is str:
        for key in contractions_dict:
            value = contractions_dict[key]
            tweet = tweet.replace(key, value)
        return tweet
    else:
        return tweet
    
def dataset_load(filepath):
    df = pd.read_csv(filepath, sep='\t', header=None)
    return df

def preprocess_tweets(tweet):
    
    #Convert to lower case
    tweet = tweet.lower()    
    
    #Remove usermentions
    r = re.findall("@[\w]*", tweet)
    for word in r:
        tweet = re.sub(word, "usermention", tweet)
    
    #remove hashtags
    tweet = re.sub(r'#([a-z0-9]+)(?:\b|$)', "hashtag", tweet)    
    
    # remove special characters, numbers and punctuations
    tweet = re.sub(r"[^a-zA-Z]", " ", tweet)
    
    #remove shortwords
    tweet = " ".join([w for w in tweet.split() if len(w)>1])
    
    #Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", " ", tweet, flags = re.MULTILINE)
    
    #Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    stopwords_english = stopwords.words('english')
    filtered_words = [word for word in tweet_tokens if word not in stopwords_english]
    
    #Lemmatizing
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in filtered_words]
    
    return " ". join(lemma_words)

df = dataset_load(r"C:/Users/harsh/Downloads/Warwick/NLP/Assignment/semeval-tweets.tar/semeval-tweets/twitter-dev-data.txt")
df = df.rename({0: 'Id', 1: 'label' , 2: 'tweet'}, axis='columns')

df['clean_tweet'] = np.vectorize(convert_contraction_to_original)(df['tweet'])
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: preprocess_tweets(x))
df.head()

,Id,label,tweet,clean_tweet
0,262696992176304465,neutral,@Oirisheye Hey you! I'm gonna be in Dublin in ...,usermention hey gon na dublin february know sa...
1,410734138242126311,positive,Literally so excited I'm going to a Sam Smith ...,literally excited going sam smith concert october
2,237615985571058688,neutral,@WINDmobile Will there be an option to buy the...,usermention option buy gb ram model moto rd ge...
3,90473590077188360,neutral,Our Little Ms. Philippines. 👏👏👏#littleMsPhilip...,little ms philippines hashtag hashtag co kqk
4,450236582392850660,negative,"@AngryRaiderFan I know. This, TPP, expanded wa...",usermention know tpp expanded wars drone strik...


In [2]:
SEED = 2021

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

df1 = df[df.clean_tweet != '']
df1.to_csv('df.csv',index=False)

TEXT = torchtext.legacy.data.Field(tokenize = 'spacy',batch_first=True,include_lengths=True)
LABEL = torchtext.legacy.data.LabelField(dtype=torch.float,batch_first=True)
fields = [('Id', None),('label', LABEL),('tweet',None),('clean_tweet', TEXT)]

training_data=torchtext.legacy.data.TabularDataset(path = 'df.csv',format = 'csv',fields = fields,skip_header = True)

train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

TEXT.build_vocab(train_data, max_size = 4998, min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

C:\Users\harsh\anaconda3\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


defaultdict(None, {'neutral': 0, 'positive': 1, 'negative': 2})


In [3]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.clean_tweet),
    sort_within_batch=True,
    device = device)

import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        for param in self.fc.parameters():
            param.requires_grad = False

        with torch.no_grad():
            self.fc.eval()
            #print(self.fc.weight.requires_grad)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs
    
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = False, dropout = dropout)

#architecture
print(model)

#No. of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

False
classifier(
  (embedding): Embedding(1194, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 145,000 trainable parameters
torch.Size([1194, 100])


In [4]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters(),lr = 0.005)
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [5]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.clean_tweet   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze(1)  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.clean_tweet
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze(1)
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [6]:
N_EPOCHS = 100
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.534 | Train Acc: 36.08%
	 Val. Loss: 0.639 |  Val. Acc: 33.28%
	Train Loss: 0.404 | Train Acc: 41.33%
	 Val. Loss: 0.672 |  Val. Acc: 41.04%
	Train Loss: 0.181 | Train Acc: 46.84%
	 Val. Loss: 0.715 |  Val. Acc: 43.33%
	Train Loss: 0.027 | Train Acc: 54.91%
	 Val. Loss: 0.580 |  Val. Acc: 48.23%
	Train Loss: -0.119 | Train Acc: 59.20%
	 Val. Loss: 0.791 |  Val. Acc: 43.49%
	Train Loss: -0.262 | Train Acc: 63.27%
	 Val. Loss: 0.674 |  Val. Acc: 49.69%
	Train Loss: -0.315 | Train Acc: 66.16%
	 Val. Loss: 0.838 |  Val. Acc: 46.72%
	Train Loss: -0.316 | Train Acc: 65.18%
	 Val. Loss: 0.793 |  Val. Acc: 46.93%
	Train Loss: -0.357 | Train Acc: 65.62%
	 Val. Loss: 0.675 |  Val. Acc: 49.90%
	Train Loss: -0.453 | Train Acc: 69.13%
	 Val. Loss: 0.761 |  Val. Acc: 48.07%
	Train Loss: -0.517 | Train Acc: 71.03%
	 Val. Loss: 0.825 |  Val. Acc: 46.98%
	Train Loss: -0.519 | Train Acc: 70.74%
	 Val. Loss: 0.862 |  Val. Acc: 45.47%
	Train Loss: -0.552 | Train Acc: 72.39%
	 Val. Loss: 0.8